### Installing dependencies

In [ ]:
! pip install transformers[torch] sentencepiece datasets evaluate seqeval huggingface_hub wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-a

In [ ]:
! huggingface-cli login
! wandb login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https

### Importing dependencies and setting paths

In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive/")
os.chdir("/content/drive/MyDrive/Colab Notebooks/NER/")

os.environ["WANDB_PROJECT"]="DeBERTa-v3-NER"
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="false"

Mounted at /content/drive/


In [ ]:
import numpy as np
from tqdm.notebook import tqdm, trange
from spacy import displacy
from datasets import load_dataset, load_from_disk
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)
from transformers import DataCollatorForTokenClassification
import evaluate
import torch

from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

In [ ]:
DATASET_NAME = "Babelscape/multinerd"
BASE_MODEL_NAME = "microsoft/deberta-v3-base"
OUTPUT_DIR = "./deberta-v3-base-ner-B"

label2id = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-DIS": 9,
    "I-DIS": 10,
}

label_map = {
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    8: 8 ,
    13: 9,
    14: 10,
}

id2label = {v: k for k, v in label2id.items()}

label_list = [id2label[i] for i in range(len(label2id))]

saved_ner_tags = [0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14]

### Loading and filtering dataset

In [ ]:
dataset = load_dataset(DATASET_NAME)
dataset = dataset.filter(lambda x: x["lang"] == "en", num_proc=10)
# dataset.save_to_disk("./data/multinerd_en")
# dataset = load_from_disk("./data/multinerd_en")
dataset

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

Filter (num_proc=10):   0%|          | 0/2678400 [00:00<?, ? examples/s]

Filter (num_proc=10):   0%|          | 0/334800 [00:00<?, ? examples/s]

Filter (num_proc=10):   0%|          | 0/335986 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32908
    })
})

In [ ]:
def filter_ner_tags(x):
    x["ner_tags"] = [label_map.get(i, 0) for i in x["ner_tags"]]
    return x

dataset = dataset.map(filter_ner_tags)

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

### Preprocessing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

In [ ]:
def tokenize_and_preserve_tags(example, tokenizer, label_field):
    encoded = tokenizer(example["tokens"], is_split_into_words=True)
    encoded.update(example)

    word_ids = encoded.word_ids()
    labels = []
    for i in range(len(word_ids)):
        label = None
        if word_ids[i] is not None:
            label = example[label_field][word_ids[i]]
        if label is None:
            labels.append(-100)
        elif word_ids[i - 1] != word_ids[i]:
            labels.append(label)
        else:
            labels.append((label + 1) // 2 * 2)
    encoded["labels"] = labels
    encoded["word_ids"] = word_ids
    for key in ["input_ids", "attention_mask", "token_type_ids", "labels"]:
        encoded[key] = torch.LongTensor(encoded[key])
    return encoded

In [ ]:
dataset = dataset.map(lambda x: tokenize_and_preserve_tags(x, tokenizer, "filtered_ner_tags"))
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'], output_all_columns=True)
dataset

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang', 'filtered_ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang', 'filtered_ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang', 'filtered_ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 32908
    })
})

### Loading pretrained model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(BASE_MODEL_NAME, num_labels=len(label2id), id2label=id2label, label2id=label2id)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

def preprocess_logits_for_metrics(logits, labels):
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids

### Fine-tuning

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    report_to="wandb",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=(1 / 3 / 8),
    fp16=True,
    save_steps=(1 / 3 / 2),
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

In [ ]:
import wandb

wandb.init()

In [ ]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
257,0.033600,0.034458,0.924509,0.938560,0.931482,0.988525
514,0.030900,0.029589,0.916089,0.962434,0.938689,0.989192
771,0.028100,0.025052,0.931594,0.953879,0.942604,0.990752
1028,0.022100,0.025950,0.938064,0.958777,0.948308,0.991027
1285,0.023400,0.026023,0.931813,0.964035,0.947650,0.990383
1542,0.017700,0.024837,0.933135,0.966472,0.949511,0.990874
1799,0.021300,0.022810,0.952182,0.959327,0.955741,0.991820
2056,0.025200,0.023340,0.951657,0.956842,0.954242,0.991749
2313,0.014300,0.025615,0.949090,0.964059,0.956516,0.991800
2570,0.013200,0.024747,0.953603,0.962673,0.958116,0.992096


TrainOutput(global_step=6153, training_loss=0.018427644880945927, metrics={'train_runtime': 8028.3435, 'train_samples_per_second': 98.112, 'train_steps_per_second': 0.766, 'total_flos': 2.1005260134144e+16, 'train_loss': 0.018427644880945927, 'epoch': 3.0})

### Testing

In [ ]:
trainer.evaluate(dataset["test"])

{'eval_loss': 0.029370391741394997,
 'eval_precision': 0.965979937439327,
 'eval_recall': 0.9750881930229519,
 'eval_f1': 0.9705126954712443,
 'eval_accuracy': 0.99289667477176,
 'eval_runtime': 135.7927,
 'eval_samples_per_second': 242.34,
 'eval_steps_per_second': 15.148}

In [ ]:
def align_tags_with_words(token_tags, word_indices):
    word_tags = []
    for i in range(len(token_tags)):

        if token_tags[i] == -100:
            continue
        elif not word_tags or word_indices[i] != word_indices[i - 1]:
            word_tags.append(token_tags[i])
        elif word_tags[-1] == 0 and token_tags[i] != 0:
            word_tags[-1] = token_tags[i]

    for i in range(1, len(word_tags)):
        if (
            word_tags[i] > 0 and word_tags[i] % 2 == 0 and
            word_tags[i] - word_tags[i - 1] not in [0, 1]
        ):
            word_tags[i] -= 1
    return word_tags

In [ ]:
preds = trainer.predict(dataset["test"])

pred_ner_tags = []
true_ner_tags = []
for i in trange(len(dataset["test"])):
    word_ids = dataset["test"][i]["word_ids"][1:-1]
    pred = preds.predictions[i][1:len(word_ids) + 1]
    pred = [label_map.get(x, 0) for x in pred]
    pred_ner_tags.append([id2label[x] for x in align_tags_with_words(pred, word_ids)])
    true_ner_tags.append([id2label[x] for x in dataset["test"][i]["ner_tags"]])

  0%|          | 0/32908 [00:00<?, ?it/s]

In [ ]:
print(classification_report(true_ner_tags, pred_ner_tags, digits=3))

              precision    recall  f1-score   support

        ANIM      0.747     0.804     0.774      3208
         DIS      0.785     0.843     0.813      1518
         LOC      0.995     0.995     0.995     24048
         ORG      0.982     0.985     0.984      6618
         PER      0.994     0.996     0.995     10530

   micro avg      0.967     0.976     0.971     45922
   macro avg      0.901     0.925     0.912     45922
weighted avg      0.969     0.976     0.972     45922



### Inference

In [ ]:
import numpy as np
from spacy import displacy
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

FINETUNED_MODEL = "alexeyak/deberta-v3-base-ner-B"

In [ ]:
def align_tags_with_words(token_tags, word_indices):
    word_tags = []
    for i in range(len(token_tags)):
        if token_tags[i] == -100:
            continue
        elif not word_tags or word_indices[i] != word_indices[i - 1]:
            word_tags.append(token_tags[i])
        elif word_tags[-1] == 0 and token_tags[i] != 0:
            word_tags[-1] = token_tags[i]

    for i in range(1, len(word_tags)):
        if (
            word_tags[i] > 0 and word_tags[i] % 2 == 0 and
            word_tags[i] - word_tags[i - 1] not in [0, 1]
        ):
            word_tags[i] -= 1
    return word_tags

def ner_render(tokens, ner_tags, title=None, **kwargs):
    pos = 0
    ents = []
    for word, tag_idx in zip(tokens, ner_tags):
        tag = id2label[tag_idx]
        if tag.startswith('B'):
            ents.append({
                "start": pos,
                "end": pos + len(word),
                "label": tag.split("-")[1]
            })
        elif tag.startswith('I'):
            ents[-1]["end"] = pos + len(word)
        pos += (len(word) + 1)
    displacy.render({
        "text": " ".join(tokens),
        "ents": ents,
        "title": title
    }, style="ent", manual=True, jupyter=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL)
model = AutoModelForTokenClassification.from_pretrained(FINETUNED_MODEL)

In [ ]:
text = "Four days later, he scored his first Premier League goal of the season, assisting Fernando Torres for the opener and then scoring directly from a free-kick in a 2–1 away win against Arsenal at the Emirates Stadium."
# text = "Takuya Takagi scored the winner in the 88th minute, rising to head a Hiroshige Yanagimoto cross towards the Syrian goal which goalkeeper Salem Bitar appeared to have covered but then allowed to slip into the net."
inputs = tokenizer(text.split(), is_split_into_words=True, return_tensors="pt")
with torch.no_grad():
    token_tags = model(**inputs).logits[0].numpy().argmax(-1)

ner_tags = align_tags_with_words(token_tags[1:-1], inputs.word_ids()[1:-1])

In [ ]:
ner_render(text.split(), ner_tags)